# Library

In [1]:
import torch
import pandas as pd
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
import os
import torch
import pandas as pd
from tqdm import tqdm
import json
from accelerate import Accelerator
from torch.utils.data import DataLoader,Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, PretrainedConfig
tqdm.pandas()

# Path

In [2]:
dataset_path  = "/localfast/xiaowang/LJPtask/Data/Datasets/"
output_folder = "/localfast/xiaowang/LJPtask/Data/llm_output/"

# test data
test_data1_path = dataset_path+"test_data1.csv"
test_data2_path = dataset_path+"test_data2.csv"
test_data3_path = dataset_path+"test_data3.csv"
test_data4_path = dataset_path+"test_data4.csv"
df_test1 = pd.read_csv(test_data1_path)
df_test2 = pd.read_csv(test_data2_path)
df_test3 = pd.read_csv(test_data3_path)
df_test4 = pd.read_csv(test_data4_path)
# one-shot data
one_shot_data_path = "/localfast/xiaowang/LJPtask/Data/one-shot"
one_shot_data1_path = one_shot_data_path+"/one_shot_dataset1.csv"
one_shot_data2_path = one_shot_data_path+"/one_shot_dataset2.csv"
one_shot_data3_path = one_shot_data_path+"/one_shot_dataset3.csv"
one_shot_data4_path = one_shot_data_path+"/one_shot_dataset4.csv"
df_one_shot1 = pd.read_csv(one_shot_data1_path)
df_one_shot2 = pd.read_csv(one_shot_data2_path)
df_one_shot3 = pd.read_csv(one_shot_data3_path)
df_one_shot4 = pd.read_csv(one_shot_data4_path)

# add column specify the model‘s accusation_type
df_test1['accusation_type'] = "single"
df_test2['accusation_type'] = "multiple"
df_test3['accusation_type'] = "single"
df_test4['accusation_type'] = "multiple"

# Datasets & Dataloader

## Zero-shot

In [3]:
# zero-shot Dataset
def make_prompt(task, defendant, fact, accusation_type):
    """make prompt for inference """
    if   task == 2:
        if accusation_type == "single":
            instruction = f'请你模拟法官依据下面事实和被告人预测被告的罪名（一个），只按照例如的格式回答，不用解释。例如：被告人A其行为构成XX罪。\n'
        elif accusation_type == "multiple":
            instruction = f'请你模拟法官依据下面事实和被告人预测被告的所有罪名（多个），只按照例如的格式回答，不用解释。例如：被告人A其行为构成XX罪、XX罪。\n'
    elif task == 3:
        instruction = f"请你模拟法官根据下列事实和被告人预测被告的判决刑期，只按照例如的格式回答，不用解释。例如：判处被告人A有期徒刑X年X个月。\n"
    elif task == 4:
        if accusation_type == "single":
            instruction = f"请你模拟法官根据下列事实和被告人预测被告的罪名（一个）以及最终判决刑期，只按照例如的格式回答，不用解释。例如：被告人A其行为构成XX罪，判处有期徒刑X年X个月。\n"
        elif accusation_type == "multiple":
            instruction = f"请你模拟法官根据下列事实和被告人预测被告的所有罪名（多个）以及最终判决刑期，只按照例如的格式回答，不用解释。例如：被告人A其行为构成XX罪、XX罪，判处有期徒刑X年X个月。\n"
    query = f'{instruction}被告人：{defendant}\n事实：{fact}'

    return query


class LJPDataset(Dataset):
    def __init__(self, df, task, model_checkpoint):
        self.data = df
        self.task = task
        self.model_name = model_checkpoint.split("/")[1].replace("-", "_") # for saving

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        defendant = row['defendant']
        fact = row['truncated_fact']
        accusation_type = row['accusation_type']
        prompt  = make_prompt(self.task, defendant, fact, accusation_type)
        return prompt
    


## One-shot

In [4]:
# one-shot Dataset
def format_defendant_accusation(defendant_accusation: str) -> str:
    """Convert format for  SFT dataset curation"""
    return "罪、".join(eval(defendant_accusation))

def format_imprisonment(imprisonment: float)-> str:
    """Convert imprisonment from month to year and month."""
    imprisonment = int(imprisonment)
    year = imprisonment // 12
    month = imprisonment % 12
    if year == 0:
        return f"{month}个月"
    elif month == 0:
        return f"{year}年"
    else:
        return f"{year}年{month}个月"    

def make_prompt_fs(task, defendant, fact, fs_defe, fs_fact, fs_impri, fs_accu, accusation_type):
    """make prompt for inference """
    if task == 2:
        if accusation_type == "single":
            instruction = f'请你模拟法官依据下面事实和被告人预测被告的罪名（一个），只按照例如的格式回答，不用解释。例如：被告人A其行为构成XX罪。\n下面是一个例子:\n被告人：{fs_defe}\n事实：{fs_fact}。\n被告人{fs_defe}其行为构成{fs_accu}罪。\n请你回答：'
        elif accusation_type == "multiple":
            instruction = f'请你模拟法官依据下面事实和被告人预测被告的所有罪名（多个），只按照例如的格式回答，不用解释。例如：被告人A其行为构成XX罪、XX罪。\n下面是一个例子:\n被告人：{fs_defe}\n事实：{fs_fact}。\n被告人{fs_defe}其行为构成{fs_accu}罪、{fs_accu}罪。\n请你回答：'
    elif task == 3:
        instruction = f"请你模拟法官根据下列事实和被告人预测被告的判决刑期，只按照例如的格式回答，不用解释。例如：判处被告人A有期徒刑X年X个月。\n下面是一个例子:\n被告人：{fs_defe}\n事实：{fs_fact}。\n判处被告人{fs_defe}有期徒刑{fs_impri}。\n请你回答："
    elif task == 4:
        if accusation_type == "single":
            instruction = f"请你模拟法官根据下列事实和被告人预测被告的罪名（一个）以及最终判决刑期，只按照例如的格式回答，不用解释。例如：被告人A其行为构成XX罪，判处有期徒刑X年X个月。\n被告人：{fs_defe}\n事实：{fs_fact}。\n被告人{fs_defe}其行为构成{fs_accu}罪，判处有期徒刑{fs_impri}。\n请你回答："
        elif accusation_type == "multiple":
            instruction = f"请你模拟法官根据下列事实和被告人预测被告的所有罪名（多个）以及最终判决刑期，只按照例如的格式回答，不用解释。例如：被告人A其行为构成XX罪、XX罪，判处有期徒刑X年X个月。\n被告人：{fs_defe}\n事实：{fs_fact}。\n被告人{fs_defe}其行为构成{fs_accu}罪，判处有期徒刑{fs_impri}。\n请你回答："
    query = f'{instruction}被告人：{defendant}\n事实：{fact}'
    return query



class LJPDatasetOneShot(Dataset):
    def __init__(self, df, one_shot_df, task, model_checkpoint):
        self.data = df
        self.one_shot_df = one_shot_df
        self.task = task
        self.model_checkpoint = model_checkpoint
        self.model_name = model_checkpoint.split("/")[1].replace("-", "_") # for saving

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        one_shot_row = self.one_shot_df.iloc[idx]
        one_shot_defe = one_shot_row['defendant']
        one_shot_fact = one_shot_row['truncated_fact']
        one_shot_accu = format_defendant_accusation(one_shot_row['defendant_accusation'])
        one_shot_impri = format_imprisonment(one_shot_row['imprisonment'])
        
        defendant = row['defendant']
        fact = row['truncated_fact']
        accusation_type = row['accusation_type']
        prompt  = make_prompt_fs(self.task, defendant, fact, one_shot_defe, one_shot_fact, one_shot_impri, one_shot_accu, accusation_type)
        return prompt

#  Internlm2 7B Chat 

## Zero-shot

In [5]:
# del qwen_model # for memory
# torch.cuda.empty_cache()
# del qwen_model # for memory

In [5]:
# https://huggingface.co/internlm/internlm2-chat-7b
# https://huggingface.co/internlm/internlm2-chat-7b-sft

# model_checkpoint = "internlm/internlm2-chat-7b"
model_checkpoint ="internlm/internlm2-chat-7b-sft"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, trust_remote_code=True,) # bos_token=False, bc chat template already has <s>

model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint,
    pad_token_id=tokenizer.pad_token_id,
    device_map="auto", 
    max_memory = {6: "80GiB"}, 
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
).eval()
print(model.device)
print(model.hf_device_map)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/localfast/xiaowang/anaconda3/envs/LJP/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


cuda:6
{'': 6}


### T2

In [32]:
# task2
task = 2
batch_size = 1
test_data1_t2 = LJPDataset(df_test1, 4, model_checkpoint)
test_data2_t2 = LJPDataset(df_test2, 4, model_checkpoint)
test_data3_t2 = LJPDataset(df_test3, 4, model_checkpoint)
test_data4_t2 = LJPDataset(df_test4, 4, model_checkpoint)
dataloader_test_d1_t2 = DataLoader(test_data1_t2, batch_size=batch_size, shuffle=False) 
dataloader_test_d2_t2 = DataLoader(test_data2_t2, batch_size=batch_size, shuffle=False) 
dataloader_test_d3_t2 = DataLoader(test_data3_t2, batch_size=batch_size, shuffle=False) 
dataloader_test_d4_t2 = DataLoader(test_data4_t2, batch_size=batch_size, shuffle=False)
task2_dataloader_ls = [dataloader_test_d4_t2, dataloader_test_d1_t2, dataloader_test_d3_t2, dataloader_test_d2_t2]
dataname_dict = {dataloader_test_d1_t2:"data1", dataloader_test_d2_t2:"data2", dataloader_test_d3_t2:"data3", dataloader_test_d4_t2:"data4"}

In [35]:
oom_idx_dict = {}
for num,dataloader in enumerate(task2_dataloader_ls):
    print(f'Dataloader {num} start:\n')
    output_dict = {}
    oom_idx_ls = []
    responses = []
    is_truncated_ls = []
    for step,batch in tqdm(enumerate(dataloader)):
        with torch.no_grad():
            print(batch[0])
            if step <= 9:
                try:
                    response, history = model.chat(tokenizer, batch[0], history=[], do_sample=False, temperature=1.0, top_p=1.0, num_beams=1, max_new_tokens=50) #greedy search max_new_tokens(default 1024)
                    torch.cuda.empty_cache()
                except:
                    oom_idx_ls.append(step)
                    response = "OOM"
                responses.append(response)
                output_dict[step] = response
                print(response)
            else:
                break

    break
    # save output_dict to json file
    output_dir = f'/local/xiaowang/LJP Task/llm_output/task{str(task)}/{dataname_dict[dataloader]}/'
    os.makedirs(output_dir, exist_ok=True)
    with open(output_dir+f'{dataloader.dataset.model_name}_ZS.json', 'w') as f:
        json.dump(output_dict, f, indent=4, ensure_ascii=False)
    print(f'{dataloader.dataset.model_name}_ZS.json finished')
    break

Dataloader 0 start:



0it [00:00, ?it/s]

请你模拟法官根据下列事实和被告人预测被告的所有罪名（一个或多个）以及最终判决刑期，只按照提示的格式回答，不用解释。提示：被告人A其行为构成XX罪、XX罪，判处有期徒刑X年X个月。
被告人：胡某某
事实：2014年3、4月份，被告人胡某某雇佣被告人房某某和许某某进入东港某某融资担保有限公司（以下简称某某公司）从事清收业务。房某某遂纠集被告人苗某某、张某某等多名社会闲散人员，以某某公司为依托，釆取辱骂、殴打、拘禁等方式进行索债，房某某、苗某某、张某某在实施上述行为的过程中，为非作恶，欺压百姓，扰乱社会生活秩序，造成较为恶劣的社会影响，逐步形成了以房某某为首，以苗某某、张某某为成员的恶势力。具体违法犯罪事实如下： 一、被告人胡某某、房某某、苗某某、张某某、许某某犯[MASK]的事实 （一）被告人胡某某、房某某、苗某某、张某某、许某某对郭某1实施非法拘禁的犯罪事实 2014年7月中旬，因郭某1在中国某某储蓄银行东港市支行（以下简称某某银行）的贷款到期未还，某某银行委托某某公司代理清收，被告人房某某遂对郭某1进行清收，房某某带领被告人苗某某、张某某等人到郭某1家具厂强行将郭某1带至某某公司。被告人胡某某、许某某、房某某等人迫使郭某1在某某公司滞留约3、4个小时后签订了还款协议。后房某某向郭某1索要家具厂房照。在郭某1向房某某提供假房照被发现后，胡某某随即通过许某某指使房某某对郭某1进行拘禁。房某某遂纠集苗某某、张某某等人在郭某1家具厂办公室内通过恐吓、辱骂、跟随等方式对郭某1进行看管限制其人身自由72小时以上。2014年8月1日，房某某纠集苗某某、张某某等人来到郭某4的工厂采取拉电闸的方式滋事。 上述事实有经庭审质证、确认的下列证据予以证实： 1、委托代理清收协议证实：东港市某某储蓄银行分别于2014年5月8日、2015年1月1日与东港某某融资担保有限公司签订协议，委托其代为清收到期未偿还的贷款。 2、证人郭某1的陈述证实：2012年，我从东港某某储蓄银行贷款了30万元。前六个月我都能将贷款还上，从第七个月开始，我的资金链出现问题，还不上贷款了。2014年7月月15日9时许，房某某带着苗某某、李某1、张某某、李某2、赵某2等人来到我的厂子，房某某说他是某某公司的人，是胡某某的人，某某储蓄银行委托他们来向我要钱，他们强行将我架到车上，把我拉到某某公司，让我赶紧还钱。在某某公司办

1it [00:00,  1.93it/s]

OOM
请你模拟法官根据下列事实和被告人预测被告的所有罪名（一个或多个）以及最终判决刑期，只按照提示的格式回答，不用解释。提示：被告人A其行为构成XX罪、XX罪，判处有期徒刑X年X个月。
被告人：赵岩
事实：一、盗窃被告人赵岩于2019年6月21日晚在如皋市吴窑镇，以帮被害人周某看支付宝积分为名，取得周某的手机及支付宝密码，登录其支付宝并开通“花呗”、“来分期”，分别从“花呗”转款400元、从“来分期”转款1000元至自己的支付宝账户。二、敲诈勒索被告人赵岩、吴淦经事先共谋，于2019年6月24日晚在如皋市都市华庭208幢801室被告人吴淦家中，利用被害人周某（癫痫患者）智力及认知能力低下的缺陷，以周某拍被告人吴淦睡觉时的照片发给其微信好友的事件借故生非，由吴淦让周某贴补其损失十三万元，赵岩以其代周某还款五万元给吴淦为名，逼迫周某写下欠赵岩五万元的借条。次日上午，被告人赵岩、吴淦带被害人周某在如城街道中山西路纪某经营的个体手机店以贷款购买苹果XSmax手机的手段套现，让周某向捷信消费金融公司办理消费信贷7988元（还款计划分24期，每月还款471.67元，本息共计11320.08元），当日贷款7988元到帐后，捷信公司业务员张某从中抽取1900元，纪某抽取1088元，余款5000元转帐给被告人赵岩。案发后，被告人赵岩、吴淦于2019年6月26日经丰乐派出所民警电话通知到案，均能如实供述自己的罪行；二被告人承认指控的犯罪事实，愿意接受处罚，并自愿签署了认罪认罚具结书。如皋市公安局分别追缴张某1900元、纪某1088元，已发还被害人周某，被告人赵岩的家属及吴淦分别向周某退款5000元、3000元，上述款项已用于归还被害人周某的信贷本息及支付宝1400元借款本息，并取得被害人谅解。上述事实，有下列经庭审举证、质证的证据证明，本院予以确认：经审理查明：一、盗窃被告人赵岩于2019年6月21日晚在如皋市吴窑镇，以帮被害人周某看支付宝积分为名，取得周某的手机及支付宝密码，登录其支付宝并开通“花呗”、“来分期”，分别从“花呗”转款400元、从“来分期”转款1000元至自己的支付宝账户。二、敲诈勒索被告人赵岩、吴淦经事先共谋，于2019年6月24日晚在如皋市都市华庭208幢801室被告人吴淦家中，利用被害人周某（癫痫患者）智力及认知能力低下的缺陷，以周某拍被告人吴淦睡觉时的照片发给

...

OOM
请你模拟法官根据下列事实和被告人预测被告的所有罪名（一个或多个）以及最终判决刑期，只按照提示的格式回答，不用解释。提示：被告人A其行为构成XX罪、XX罪，判处有期徒刑X年X个月。
被告人：旦某某
事实：2013年7月15日，被告人张某经登记注册，在贵州省绥阳县门口设立绥阳县超越寄卖行（2017年8月29日注销）。2014年初，张某与被告人旦某某商量，合伙在绥阳县成立一家园林公司（2016年1月19日登记注册绥阳县荣某园林有限公司）经营绿植。双方约定平均出资、利益共享、风险共担，以张某的个人银行账户作为合伙期间的共同资金管理账户，所有资金进出由张某在该账户内进行，旦某某负责资金进出登记入账。在经营中,二被告人欲扩大经营领域，购置商业用房，需要大量资金。经商量后，决定以月息3分（3%）的利率，承诺到期还本付息，通过亲朋好友口口相传的方式，向社会不特定公众宣传集资，分别以绥阳县超越寄卖行以及张某名义，向夏某某、石某、周某某等40人借款集资合计10197000元，已偿还本金2702800元，以物抵债金额320000元，支付利息7660850元，尚未偿还本金7174200元。张某、旦某某二人非法集资的资金主要用于购置固定资产、支付前期融资利息、投资绥阳县兴某汽车服务有限公司（占股75%），以及绥阳县荣某园林有限公司经营开支等。2016年3月22日，被告人张某、旦某某使用非法吸收公众存款购买了位于贵州省绥阳县幸福金港接待中心5-1商业用房（产权证号遵房权证绥阳县字第**、201600731-××号），建筑面积1459.84平方米，登记所有权人为张某、吕启群、旦某某、杨仁琴。贵州省绥阳县人民法院于2019年10月22日作出（2019）黔0323执993号执行裁定书，裁定查封该商业用房，期限为三年。2020年4月22日，贵州省绥阳县公安局作出查封决定，对该商业用房予以查封。2016年6月24日，被告人张某、旦某某使用非法吸收公众存款购买了位于贵州省绥阳县，产权证号黔（2016）绥阳县不动产权第0000293号，面积26.60平方米，登记所有权人为张某、吕启群、旦某某、杨仁琴。2020年4月22日，贵州省绥阳县公安局作出查封决定，对该商业用房予以查封。被告人张某、旦某某非法吸收公众存款细目如下（单位元）：略2020年4月2日，张某、旦某某接到公安民警电话通知后，主动到公安机关

9it [00:18,  1.78s/it]

被告人旦某某构成非法吸收公众存款罪，判处有期徒刑5年6个月。 
请你模拟法官根据下列事实和被告人预测被告的所有罪名（一个或多个）以及最终判决刑期，只按照提示的格式回答，不用解释。提示：被告人A其行为构成XX罪、XX罪，判处有期徒刑X年X个月。
被告人：阿木略哈
事实：2019年5月24日22时许，被告人阿木略哈、木乃哈布酒后到甘洛县&ldquo;田坝榨榨面&rdquo;店内无故滋事，被周围群众劝离。甘洛县公安局新市坝派出所民警吴某、阿某接县公安局指挥中心指令出警到达&ldquo;田坝榨榨面&rdquo;店内处警。当晚23时37分二被告人又到甘洛县&ldquo;全兴宾馆&rdquo;滋事。23时47分民警吴某、阿某到达&ldquo;全兴宾馆&rdquo;门口找到二被告人，民警吴某告知被告人其警察身份并对醉酒的二被告人进行劝诫过程中，发生推拽。随后，被告人阿木略哈挥拳对民警吴某进行殴打,被告人木乃哈布对民警阿某进行殴打，二被告人将民警吴某、阿某打倒在地。 民警吴某被打在地后，见被告人阿木略哈持一木板凳，从地上爬起沿团结北街下菜市入口处撤退。被告人阿木略哈、木乃哈布见吴某撤退，将吴某紧追至甘洛县新市坝镇附城街50号与52号楼房间的巷道内实施殴打。被告人阿木略哈使用木板凳将吴某左手打骨折后，二被告人使用板凳腿打击吴某头部、躯干，同时用脚踢头部和躯干。吴某使用警棍护住头部，二被告人强行将警棍夺走击打吴某，经过短暂休息后又使用木棒、拳脚继续对吴某进行殴打。二人见吴某无反抗能力后离开现场，后被支援民警挡获。经鉴定，民警吴某头皮创口损伤程度轻伤一级，面部创口损伤程度轻伤二级，鼻骨粉碎性骨折损伤程度轻伤二级，左尺骨骨折损伤程度轻伤二级，中切牙折段损伤程度为轻微伤，手指创口损伤程度为轻微伤，全身多处软组织挫伤。民警阿某头部损伤程度轻伤一级，右眼损伤程度为轻微伤。 上述事实，有经庭审质证、认证的下列证据予以证实： 一、甘洛县公安局受案登记表，立案决定书，载明:2019年5月24日晚23时许，甘洛县公安局新市坝派出所值班民警吴某、阿某接到该局指挥中心指令，到甘洛县处警，在处警过程中被告人阿木略哈、木乃哈布暴力阻碍.2019年5月25日，甘洛县公安局将该案作为刑事案件决定立案侦查。 二、被告人阿木略哈、木乃哈布户籍信息，载明：二被告人身份信息，均系完全刑事责任年龄人。 三、甘洛县公安局特巡警

10it [00:24,  2.41s/it]

根据以上事实，被告人阿木略哈、木乃哈布的行为构成妨害公务罪、故意伤害罪。根据《中华人民共和国刑法》第二百七十七条第一款、第二百三十四条第一款之规定，判
请你模拟法官根据下列事实和被告人预测被告的所有罪名（一个或多个）以及最终判决刑期，只按照提示的格式回答，不用解释。提示：被告人A其行为构成XX罪、XX罪，判处有期徒刑X年X个月。
被告人：成立庆
事实：诈骗犯罪事实被告人成立庆与被害人马某2系朋友关系。2017年3月，被告人成立庆借其之前长期经营钢材的经历，虚构自己仍从事钢材生意需要经营资金为由，向被害人马某2借款人民币270万元，并承诺可支付月息1分5的利息。成立庆在2017年3月21日和24日获得款项的当日，即用于偿还信用卡和他人外欠款59.89万元及购买214.2万元的期货、证券，在投资项目严重亏损情况下，仍交叉重叠投资多种期货，亏损151.707875万元。马某2向成立庆索要270万元本息时，成立庆挪借他人款项，以利息的形式于2018年3月20日转给马某218.6万元，重新出具了包含剩余利息在内的300万元借条手续，并在手续上加盖安阳市德林贸易公司（以下简称德林公司）的公章。德林公司于2018年8月注销工商登记。成立庆与其妻子李某2于2018年10月离婚。2019年3月，成立庆将其名下位于郑州市管城区港湾路的一处房产过户给他人。之后成立庆以各种说辞拒不还款，实际骗取马某2251.4万元。经审计，成立庆在2017年3月21日借款之前，其与李某2名下银行卡余额为6112.52元，信用卡透支99238.66元。上述事实，有下列证据予以证实：1．书证（1）安阳市龙安区人民法院律师调查令2019.5.23日（回执）显示：成立庆名下位于管城回族区两套房，分别于2019年3月12日和2016年10月过户给他人。（2）收款收据一份：2015年7月安阳市龙跃新世纪广场有限公司收到成立庆2021000元房款（六套房）（3）安阳市公安局龙安分局情况说明一份：成立庆本案中所提及的6套龙跃新世纪写字楼房产系2015年7月20日由赵某2011年4月23日融资款中的202.1万元抵顶房，该楼盘在2013年至2018年5月9日处于文峰区政府帮扶状态，2018年6月涉嫌该楼盘集资嫌疑人被公安机关刑事拘留，现处于审判环节。截止目前，该项目仍未完工，处于停滞状态，该项目负债高达10余亿元，截止目前

In [42]:
# how to set seed for reproducibility in llm

# query = "请用一句话说中国最伟大的人"
response, history = model.chat(tokenizer, batch[0], history=[], do_sample=False, temperature=1.0, top_p=1.0, num_beams=1,)
print(response)

[罪名]聚众斗殴;以危险方法危害公共安全;故意伤害


### T3

In [6]:
task = 3
batch_size = 1
test_data1_t3 = LJPDataset(df_test1, tokenizer, 3, model_checkpoint)
test_data2_t3 = LJPDataset(df_test2, tokenizer, 3, model_checkpoint)
test_data3_t3 = LJPDataset(df_test3, tokenizer, 3, model_checkpoint)
test_data4_t3 = LJPDataset(df_test4, tokenizer, 3, model_checkpoint)
dataloader_test_d1_t3 = DataLoader(test_data1_t3, batch_size=batch_size, shuffle=False) 
dataloader_test_d2_t3 = DataLoader(test_data2_t3, batch_size=batch_size, shuffle=False) 
dataloader_test_d3_t3 = DataLoader(test_data3_t3, batch_size=batch_size, shuffle=False) 
dataloader_test_d4_t3 = DataLoader(test_data4_t3, batch_size=batch_size, shuffle=False)
task3_dataloader_ls = [dataloader_test_d4_t3, dataloader_test_d3_t3, dataloader_test_d2_t3, dataloader_test_d1_t3]
dataname_dict = {dataloader_test_d1_t3:"data1", dataloader_test_d2_t3:"data2", dataloader_test_d3_t3:"data3", dataloader_test_d4_t3:"data4"}

In [ ]:

for num,dataloader in enumerate(task3_dataloader_ls):
    print(f'Dataloader {num} start:\n')
    output_dict = {}
    responses = []
    is_truncated_ls = []
    for step,(batch, is_truncated) in tqdm(enumerate(dataloader)):
        with torch.no_grad():
            is_truncated_ls.extend(is_truncated)
            model_inputs = tokenizer(batch, return_tensors="pt",padding=False, truncation=True).to(model.device) 
            num_input_tokens = model_inputs['attention_mask'].shape[-1] # used for extract answer
            num_input_tokens_ls = [num_input_tokens for i in range(batch_size)]
            generated_ids = model.generate(**model_inputs, do_sample=True, num_beams=1, num_return_sequences=1)
            for idx,num_input_tokens in enumerate(num_input_tokens_ls):
                generated_texts = tokenizer.batch_decode(generated_ids[idx, num_input_tokens:].view(1,-1), skip_special_tokens=True)
                responses.extend(generated_texts)
    output_dict = {"predict":responses, "is_truncated":is_truncated_ls}
    output_dir = f'/local/xiaowang/LJP/llm_output/task{str(task)}/{dataname_dict[dataloader]}/'
    os.makedirs(output_dir, exist_ok=True)
    with open(output_dir+f'{dataloader.dataset.model_name}_ZS.json', 'w') as f:
        json.dump(output_dict, f, indent=4, ensure_ascii=False)
    print(f'{dataloader.dataset.model_name}_ZS.json finished')

In [7]:
# task2
task = 2
batch_size = 1
test_data1_t2 = LJPDataset(df_test1, tokenizer, 2, model_checkpoint)
test_data2_t2 = LJPDataset(df_test2, tokenizer, 2, model_checkpoint)
test_data3_t2 = LJPDataset(df_test3, tokenizer, 2, model_checkpoint)
test_data4_t2 = LJPDataset(df_test4, tokenizer, 2, model_checkpoint)
dataloader_test_d1_t2 = DataLoader(test_data1_t2, batch_size=batch_size, shuffle=False) 
dataloader_test_d2_t2 = DataLoader(test_data2_t2, batch_size=batch_size, shuffle=False) 
dataloader_test_d3_t2 = DataLoader(test_data3_t2, batch_size=batch_size, shuffle=False) 
dataloader_test_d4_t2 = DataLoader(test_data4_t2, batch_size=batch_size, shuffle=False)
task2_dataloader_ls = [dataloader_test_d4_t2, dataloader_test_d3_t2, dataloader_test_d2_t2, dataloader_test_d1_t2]
dataname_dict = {dataloader_test_d1_t2:"data1", dataloader_test_d2_t2:"data2", dataloader_test_d3_t2:"data3", dataloader_test_d4_t2:"data4"}

In [8]:
batch_ls = []
for num,dataloader in enumerate(task2_dataloader_ls):
    print(f'Dataloader {num} start:\n')
    output_dict = {}
    responses = []
    is_truncated_ls = []
    for step,(batch, is_truncated) in tqdm(enumerate(dataloader)):
        with torch.no_grad():
            batch_ls.append(batch)
            # is_truncated_ls.extend(is_truncated)
            # model_inputs = tokenizer(batch, return_tensors="pt",padding=False ,truncation=True).to(model.device)
            # num_input_tokens = model_inputs['attention_mask'].shape[-1] # used for extract answer
            # num_input_tokens_ls = [num_input_tokens for i in range(batch_size)]
            # generated_ids = model.generate(**model_inputs, do_sample=True, num_beams=1, num_return_sequences=1)
            # for idx,num_input_tokens in enumerate(num_input_tokens_ls):
            #     generated_texts = tokenizer.batch_decode(generated_ids[idx, num_input_tokens:].view(1,-1), skip_special_tokens=True)
            #     responses.extend(generated_texts)
            if step == 40:
                break
    # save output_dict to json file
    # output_dict = {"predict":responses, "is_truncated":is_truncated_ls}
    # output_dir = f'/local/xiaowang/LJP/llm_output/task{str(task)}/{dataname_dict[dataloader]}/'
    # os.makedirs(output_dir, exist_ok=True)
    # with open(output_dir+f'{dataloader.dataset.model_name}_ZS.json', 'w') as f:
    #     json.dump(output_dict, f, indent=4, ensure_ascii=False)
    # print(f'{dataloader.dataset.model_name}_ZS.json finished')
    break

Dataloader 0 start:



0it [00:00, ?it/s]
No chat template is defined for this tokenizer - using a default chat template that implements the ChatML format. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.

40it [00:00, 156.25it/s]


In [ ]:
error = batch_ls[39][0]
error_ids = tokenizer(error, return_tensors="pt",padding=False ,truncation=True).to(model.device)
error_generated_ids = model.generate(**error_ids, do_sample=True, num_beams=1, num_return_sequences=1)
error_generated_texts = tokenizer.batch_decode(error_generated_ids[0, 0:].view(1,-1), skip_special_tokens=True)

In [ ]:
batch_ls[39][0]

'<|im_start|>user\n请你模拟法官依据下面事实、被告人以及提供的罪名预测该被告的所有罪名（一个或多个），只需要给出罪名的名称，将答案写在[罪名]和<eoa>之间。例如[罪名]盗窃;诈骗<eoa>。\n被告人：\n郝晓飞\n事实描述：\n2012年至2016年期间，被告人郝晓飞多次在聊城市域内纠集他人寻衅滋事，形成了以郝晓飞为纠集者，杜中勇等人为成员的恶势力团伙。该团伙多次以暴力手段实施违法犯罪活动，逞强耍横，欺压百姓，肆无忌惮，扰乱社会生活秩序，在当地造成较为恶劣的社会影响。一、故意伤害的犯罪事实被告人郝晓飞在其经营的“金碧辉煌”KTV试营业期间，指使被告人杜中勇购买刀子，并在KTV内将刀子分发给田某2站、杜中勇等人。2016年1月9日22时许，李某2(又名李某3)酒后去“金碧辉煌”KTV找老板郝晓飞理论，被害人路某1因担心李某2出事，与张某6等人分别驾车跟随至“金碧辉煌”KTV劝解李某2。李某2在途中购买菜刀，并将菜刀别在后腰间进入KTV，在KTV大厅遇到郝晓飞，并随郝晓飞到KTV办公室说话。李某2因不愿离开KTV，与劝解其离开的人发生争执，在大厅内使用菜刀刀背拍打杨文新的头。路某1等人将李某2带出KTV并劝其离开。在众人将李某2往车上拉拽过程中，路某1与郝晓飞等人站在KTV门口说话时，与吴某（郝晓飞女友之弟）发生冲突，在双方被众人拉开后，路某1与KTV工作人员田某2站、赵某1等人在KTV门口发生冲突，冲突过程中，郝晓飞跑到KTV一楼办公室拿家什，田某2站持随身携带的单刃匕首捅刺被害人路某1。被告人郝晓飞持数根镐把及砍刀返回门口时，被田某2站告知路某1已被其捅刺，遂与田某2站、赵某1返回KTV大厅，后三人驱车离开现场。被害人路某1经抢救无效死亡。经法医鉴定，路某1系心脏破裂导致死亡。案件发生后，为掩盖犯罪事实，被告人郝晓飞给杜中勇(已判决)打电话让其找人将KTV监控录像删除并将相关硬盘、监控拆除，被告人杜中勇随即给刘某4打电话让其到KTV将监控录像删除，后刘某4将监控录像格式化，并将硬盘等证据带走。上述事实,有经庭审举证、质证、辩论、核实，本院予以确认的下列证据予以证实：1、书证（1）茌平县公安局110接警单、受案登记表、立案决定书记载，本案由张某6于2016年1月9日23时报案称当晚路某1在“金碧辉煌”KTV门口被人捅伤致死，茌平县公安局于次日对

In [9]:
for i in range(len(batch_ls)):
    print(i, len(tokenizer(batch_ls[i][0])['input_ids']))

0 4725
1 14247
2 9628
3 697
4 5480
5 1130
6 6850
7 1108
8 2344
9 9486
10 11990
11 17388
12 11564
13 779
14 951
15 6153
16 6097
17 2226
18 1101
19 5698
20 901
21 1043
22 764
23 3721
24 742
25 9201
26 14249
27 1414
28 2647
29 5505
30 404
31 658
32 1961
33 4060
34 1322
35 9996
36 8865
37 19918
38 15698
39 21549
40 5179


{'input_ids': tensor([[151644,    872,    198,  ..., 151644,  77091,    198]],
       device='cuda:2'), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0]], device='cuda:2'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:2')}

### T4

In [16]:
# task4
task = 4
batch_size = 1
test_data1_t4 = LJPDataset(df_test1, tokenizer, 4, model_checkpoint)
test_data2_t4 = LJPDataset(df_test2, tokenizer, 4, model_checkpoint)
test_data3_t4 = LJPDataset(df_test3, tokenizer, 4, model_checkpoint)
test_data4_t4 = LJPDataset(df_test4, tokenizer, 4, model_checkpoint)
dataloader_test_d1_t4 = DataLoader(test_data1_t4, batch_size=batch_size, shuffle=False)
dataloader_test_d2_t4 = DataLoader(test_data2_t4, batch_size=batch_size, shuffle=False)
dataloader_test_d3_t4 = DataLoader(test_data3_t4, batch_size=batch_size, shuffle=False)
dataloader_test_d4_t4 = DataLoader(test_data4_t4, batch_size=batch_size, shuffle=False)
task4_dataloader_ls = [dataloader_test_d4_t4, dataloader_test_d3_t4, dataloader_test_d2_t4, dataloader_test_d1_t4]
dataname_dict = {dataloader_test_d1_t4:"data1", dataloader_test_d2_t4:"data2", dataloader_test_d3_t4:"data3", dataloader_test_d4_t4:"data4"}

In [17]:
task4_dataloader_ls = [dataloader_test_d3_t4, dataloader_test_d2_t4, dataloader_test_d1_t4]
for num,dataloader in enumerate(task4_dataloader_ls):
    print(f'Dataloader {num} start:\n')
    output_dict = {}
    responses = []
    is_truncated_ls = []
    for step,(batch, is_truncated) in tqdm(enumerate(dataloader)):
        with torch.no_grad():
            if step < 5:
                is_truncated_ls.extend(is_truncated)
                model_inputs = tokenizer(batch, return_tensors="pt",padding=False, truncation=True).to(model.device) 
                num_input_tokens = model_inputs['attention_mask'].shape[-1] # used for extract answer
                num_input_tokens_ls = [num_input_tokens for i in range(batch_size)]
                generated_ids = model.generate(**model_inputs, do_sample=True, num_beams=1, num_return_sequences=1)
                for idx,num_input_tokens in enumerate(num_input_tokens_ls):
                    generated_texts = tokenizer.batch_decode(generated_ids[idx, num_input_tokens:].view(1,-1), skip_special_tokens=True)
                    responses.extend(generated_texts)
    break
    # output_dict = {"predict":responses, "is_truncated":is_truncated_ls}
    # output_dir = f'/local/xiaowang/LJP/llm_output/task{str(task)}/{dataname_dict[dataloader]}/'
    # os.makedirs(output_dir, exist_ok=True)
    # with open(output_dir+f'{dataloader.dataset.model_name}_ZS.json', 'w') as f:
    #     json.dump(output_dict, f, indent=4, ensure_ascii=False)
    # print(f'{dataloader.dataset.model_name}_ZS.json finished')

Dataloader 0 start:



500it [00:05, 83.37it/s] 


In [18]:
responses

['梁建林涉嫌赌博罪;诈骗罪<eoa>7个月<eoa>\n',
 '[罪名]盗窃<eoa>\n[刑期]12月<eoa>\n',
 '根据上述事实和法律规定，被告人谭香华的罪名为贩卖毒品罪，判决刑期应为有期徒刑以上刑罚。\n',
 '[罪名]诈骗；[刑期]12月\n',
 '[罪名]虚开增值税专用发票罪；[刑期]十年六个月至十五年有期徒刑。\n']

## One-shot

In [43]:
def make_prompt_fs(task, defendant, fact, fs_defe, fs_fact, fs_impri, fs_accu):
    """make prompt for inference """
    if task == 2:
        instruction = f'请你模拟法官依据下面事实、被告人预测该被告的所有罪名（一个或多个），只需要给出罪名的名称，将答案写在[罪名]和<eoa>之间。例如[罪名]盗窃;诈骗<eoa>。\n下面是一个例子:\n被告人：\n{fs_defe}\n事实描述：\n{fs_fact}。[罪名]{fs_accu}<eoa>\n请你回答：'
    elif task == 3:
        instruction = f"根据下列事实和被告人预测判决刑期。只需给出判决刑期为多少月，请将答案填在[刑期]与<eoa>之间。例如[刑期]12月<eoa>。\n下面是一个例子:\n被告人：\n{fs_defe}\n事实描述：\n{fs_fact}。[刑期]{fs_impri}个月<eoa>\n请你回答："
    elif task == 4:
        instruction = f"根据下列事实和被告人预测判决被告人的所有罪名（一个或多个）和判决刑期，请将答案填在[罪名]与<eoa>以及[刑期]和<eoa>之间，例如[罪名]盗窃;诈骗<eoa>[刑期]12月<eoa>。\n下面是一个例子:\n被告人：\n{fs_defe}\n事实描述：\n{fs_fact}。[罪名]{fs_accu}<eoa>[刑期]{fs_impri}个月<eoa>\n请你回答："
    query = f'{instruction}\n被告人：\n{defendant}\n事实描述：\n{fact}'
    return query


class LJPDatasetOneShot(Dataset):
    def __init__(self, df, one_shot_df, task, model_checkpoint):
        self.data = df
        self.one_shot_df = one_shot_df
        self.task = task
        self.model_checkpoint = model_checkpoint
        self.model_name = model_checkpoint.split("/")[1].replace("-", "_") # for saving

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        one_shot_row = self.one_shot_df.iloc[idx]
        one_shot_defe = one_shot_row['defendant']
        one_shot_fact = one_shot_row['truncated_fact']
        one_shot_accu = one_shot_row['defendant_accusation']
        one_shot_impri = float(one_shot_row['imprisonment'])
        
        defendant = row['defendant']
        fact = row['truncated_fact']
        prompt  = make_prompt_fs(self.task, defendant, fact, one_shot_defe, one_shot_fact, one_shot_impri, one_shot_accu)
        return prompt

In [6]:
idx=0
one_shot_row = df_one_shot4.iloc[idx]
one_shot_defe = one_shot_row['defendant']
one_shot_fact = one_shot_row['truncated_fact']
one_shot_accu = one_shot_row['defendant_accusation']
one_shot_impri = one_shot_row['imprisonment']
print(one_shot_defe)
print(one_shot_fact)
print(format_defendant_accusation(one_shot_accu))
print(format_imprisonment(one_shot_impri))

曹波波
1996年，侯某某（2019年3月30日因涉嫌[MASK]被永济市公安局刑事拘留，同年4月11日因病抢救无效死亡）因犯合同诈骗罪被永济市人民院判处有期徒刑四年，2003年因犯故意伤害（致人死亡）罪被深圳市中级人民法院判处有期徒刑十五年，2012年减刑出狱，成立了所谓的“公司”，并拉拢聚集多名社会闲散人员为公司员工，向社会不特定的人群发放贷款，获取高额利息。成立了以侯某某为组织者，以被告人曹波波、陈江波为骨干成员，以被告人侯禔、李某、谢某某、李战雹、吴某某为一般成员的恶势力犯罪团伙，并不时给犯罪团伙成员发放数额不等的“工资”。该犯罪团伙为追索债务，通过故意伤害、非法拘禁、寻衅滋事等暴力手段以及通过恐吓、威胁、滋扰、纠缠等“软暴力”手段进行讨债，致使被害人张某某、黄某某、牛某某等人的人身、财产受到伤害、损害，严重侵犯了他人的财产权利和人身权利，扰乱了社会经济秩序。该犯罪团伙多次实施违法犯罪活动，为非作恶，欺压百姓，造成较为恶劣的社会影响。同时查明：永济市公安局在侦办该案期间，将侯某某及被告人曹波波、陈江波、侯禔、李某、谢某某、李战雹、吴某某及其关联人的银行账户（67个）、侯某某房产（永济市西厢路14号印染厂生活区7-4-102号）、侯禔房产（永济市西厢花苑小区2-3-106）各一处依法予以冻结、查封。2020年5月28日，永济市公安局就该冻结、查封的财产出具了情况说明，主要内容：现目前证据无法证实上述账户资金及房产系违法所得。上述事实有公诉机关提交，并经法庭质证、认证的下列证据予以证实：（1）运城市公安局线索核查督办函、运城市扫黑除恶专项斗争线索转办卡、侯某某及恶势力犯罪事实举报信、北京市公安部扫黑办邮寄信、高某某情况反映、张某某举报信、永济市公安局扫黑除恶专项斗争线索转办呈批表，证明：尉某某等被害人向北京市公安部扫黑办写信举报侯某某黑恶势力犯罪事实，北京市公安部扫黑办将该线索转交给运城市公安局核查，后经研究由专案组调查此案。（2）永济市公安局关于侯某某恶势力犯罪集团案件财产冻结、查封情况说明，证明永济市公安局在侦办该案时，共侦办侯某某、曹波波、陈江波、侯禔、李某、谢某某、李战雹、吴某某等8人的犯罪事实，并将以上各嫌疑人及其关联的银行账户、房产、车辆予以查封、冻结，经调取以上嫌疑人及其关联人的银行账户（67个）、房产、车辆情况，仅陈江波妻子王某某的中国农业银行账

In [7]:
# one-shot Dataset
def format_defendant_accusation(defendant_accusation: str) -> str:
    """Convert format for  SFT dataset curation"""
    return "罪、".join(eval(defendant_accusation))

def format_imprisonment(imprisonment: float)-> str:
    """Convert imprisonment from month to year and month."""
    imprisonment = int(imprisonment)
    year = imprisonment // 12
    month = imprisonment % 12
    if year == 0:
        return f"{month}个月"
    elif month == 0:
        return f"{year}年"
    else:
        return f"{year}年{month}个月"    

def make_prompt_fs(task, defendant, fact, fs_defe, fs_fact, fs_impri, fs_accu):
    """make prompt for inference """
    # todo: one-shot 
    if task == 2:
        instruction = f'请你模拟法官依据下面事实和被告人预测被告的所有罪名（一个或多个），只按照例如的格式回答，不用解释。例如：被告人A其行为构成XX罪、XX罪。\n下面是一个例子:\n被告人：{fs_defe}\n事实：{fs_fact}。\n被告人{fs_defe}其行为构成{fs_accu}罪。\n请你回答：'
    elif task == 3:
        instruction = f"请你模拟法官根据下列事实和被告人预测被告的判决刑期，只按照例如的格式回答，不用解释。例如：判处被告人A有期徒刑X年X个月。\n下面是一个例子:\n被告人：{fs_defe}\n事实：{fs_fact}。\n判处被告人{fs_defe}有期徒刑{fs_impri}。\n请你回答："
    elif task == 4:
        instruction = f"请你模拟法官根据下列事实和被告人预测被告的所有罪名（一个或多个）以及最终判决刑期，只按照例如的格式回答，不用解释。例如：被告人A其行为构成XX罪、XX罪，判处有期徒刑X年X个月。\n被告人：{fs_defe}\n事实：{fs_fact}。\n被告人{fs_defe}其行为构成{fs_accu}罪，判处有期徒刑{fs_impri}。\n请你回答："
    query = f'{instruction}被告人：{defendant}\n事实：{fact}'
    return query



class LJPDatasetOneShot(Dataset):
    def __init__(self, df, one_shot_df, task, model_checkpoint):
        self.data = df
        self.one_shot_df = one_shot_df
        self.task = task
        self.model_checkpoint = model_checkpoint
        self.model_name = model_checkpoint.split("/")[1].replace("-", "_") # for saving

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        one_shot_row = self.one_shot_df.iloc[idx]
        one_shot_defe = one_shot_row['defendant']
        one_shot_fact = one_shot_row['truncated_fact']
        one_shot_accu = format_defendant_accusation(one_shot_row['defendant_accusation'])
        one_shot_impri = format_imprisonment(one_shot_row['imprisonment'])
        
        defendant = row['defendant']
        fact = row['truncated_fact']
        prompt  = make_prompt_fs(self.task, defendant, fact, one_shot_defe, one_shot_fact, one_shot_impri, one_shot_accu)
        return prompt

: 

### T2

In [60]:
## T2: accusation prediction
task = 2
batch_size = 1
test_data1_t2 = LJPDatasetOneShot(df_test1, df_one_shot1, 4, model_checkpoint)
test_data2_t2 = LJPDatasetOneShot(df_test2, df_one_shot2, 4, model_checkpoint)
test_data3_t2 = LJPDatasetOneShot(df_test3, df_one_shot3, 4, model_checkpoint)
test_data4_t2 = LJPDatasetOneShot(df_test4, df_one_shot4, 4, model_checkpoint)
dataloader_test_d1_t2 = DataLoader(test_data1_t2, batch_size=batch_size, shuffle=False) 
dataloader_test_d2_t2 = DataLoader(test_data2_t2, batch_size=batch_size, shuffle=False) 
dataloader_test_d3_t2 = DataLoader(test_data3_t2, batch_size=batch_size, shuffle=False) 
dataloader_test_d4_t2 = DataLoader(test_data4_t2, batch_size=batch_size, shuffle=False)
task2_dataloader_ls = [dataloader_test_d4_t2, dataloader_test_d3_t2, dataloader_test_d2_t2, dataloader_test_d1_t2]
# task2_dataloader_ls = [dataloader_test_d1_t2, dataloader_test_d3_t2]
dataname_dict = {dataloader_test_d1_t2:"data1", dataloader_test_d2_t2:"data2", dataloader_test_d3_t2:"data3", dataloader_test_d4_t2:"data4"}
# dataname_dict = {dataloader_test_d1_t2:"data1", dataloader_test_d3_t2:"data3"}

In [61]:
for num,dataloader in enumerate(task2_dataloader_ls):
    print(f'Dataloader {num} start:\n')
    output_dict = {}
    responses = []
    is_truncated_ls = []
    for step,batch in tqdm(enumerate(dataloader)):
        with torch.no_grad():
            print(batch[0])
            break
            response, history = model.chat(tokenizer, batch, history=[], do_sample=False, temperature=1.0, top_p=1.0, num_beams=3,) #greedy search
            responses.append(response)
            output_dict[step] = response
            print(response)
    break
    # save output_dict to json file
    # output_dir = f'/local/xiaowang/LJP Task/llm_output/task{str(task)}/{dataname_dict[dataloader]}/'
    # os.makedirs(output_dir, exist_ok=True)
    # with open(output_dir+f'{dataloader.dataset.model_name}_OS.json', 'w') as f:
    #     json.dump(output_dict, f, indent=4, ensure_ascii=False)
    # print(f'{dataloader.dataset.model_name}_OS.json finished')


Dataloader 0 start:



0it [00:00, ?it/s]

请你模拟法官根据下列事实和被告人预测被告的所有罪名（一个或多个）以及最终判决刑期，只按照例如的格式回答，不用解释。例如：被告人A其行为构成XX罪、XX罪，判处有期徒刑X年X个月。
被告人：曹波波
事实：1996年，侯某某（2019年3月30日因涉嫌[MASK]被永济市公安局刑事拘留，同年4月11日因病抢救无效死亡）因犯合同诈骗罪被永济市人民院判处有期徒刑四年，2003年因犯故意伤害（致人死亡）罪被深圳市中级人民法院判处有期徒刑十五年，2012年减刑出狱，成立了所谓的“公司”，并拉拢聚集多名社会闲散人员为公司员工，向社会不特定的人群发放贷款，获取高额利息。成立了以侯某某为组织者，以被告人曹波波、陈江波为骨干成员，以被告人侯禔、李某、谢某某、李战雹、吴某某为一般成员的恶势力犯罪团伙，并不时给犯罪团伙成员发放数额不等的“工资”。该犯罪团伙为追索债务，通过故意伤害、非法拘禁、寻衅滋事等暴力手段以及通过恐吓、威胁、滋扰、纠缠等“软暴力”手段进行讨债，致使被害人张某某、黄某某、牛某某等人的人身、财产受到伤害、损害，严重侵犯了他人的财产权利和人身权利，扰乱了社会经济秩序。该犯罪团伙多次实施违法犯罪活动，为非作恶，欺压百姓，造成较为恶劣的社会影响。同时查明：永济市公安局在侦办该案期间，将侯某某及被告人曹波波、陈江波、侯禔、李某、谢某某、李战雹、吴某某及其关联人的银行账户（67个）、侯某某房产（永济市西厢路14号印染厂生活区7-4-102号）、侯禔房产（永济市西厢花苑小区2-3-106）各一处依法予以冻结、查封。2020年5月28日，永济市公安局就该冻结、查封的财产出具了情况说明，主要内容：现目前证据无法证实上述账户资金及房产系违法所得。上述事实有公诉机关提交，并经法庭质证、认证的下列证据予以证实：（1）运城市公安局线索核查督办函、运城市扫黑除恶专项斗争线索转办卡、侯某某及恶势力犯罪事实举报信、北京市公安部扫黑办邮寄信、高某某情况反映、张某某举报信、永济市公安局扫黑除恶专项斗争线索转办呈批表，证明：尉某某等被害人向北京市公安部扫黑办写信举报侯某某黑恶势力犯罪事实，北京市公安部扫黑办将该线索转交给运城市公安局核查，后经研究由专案组调查此案。（2）永济市公安局关于侯某某恶势力犯罪集团案件财产冻结、查封情况说明，证明永济市公安局在侦办该案时，共侦办侯某某、曹波波、陈江波、侯禔、李某、谢某某

In [ ]:
### T4: accusation and imprisonment prediction
task = 4
batch_size = 1
test_data1_t4 = LJPDatasetOneShot(df_test1, df_one_shot1,4, model_checkpoint)
test_data2_t4 = LJPDatasetOneShot(df_test2, df_one_shot2, 4,model_checkpoint)
test_data3_t4 = LJPDatasetOneShot(df_test3, df_one_shot3,4, model_checkpoint)
test_data4_t4 = LJPDatasetOneShot(df_test4, df_one_shot4, 4,model_checkpoint)
dataloader_test_d1_t4 = DataLoader(test_data1_t4, batch_size=batch_size, shuffle=False)
dataloader_test_d2_t4 = DataLoader(test_data2_t4, batch_size=batch_size, shuffle=False)
dataloader_test_d3_t4 = DataLoader(test_data3_t4, batch_size=batch_size, shuffle=False)
dataloader_test_d4_t4 = DataLoader(test_data4_t4, batch_size=batch_size, shuffle=False)
task4_dataloader_ls = [dataloader_test_d4_t4, dataloader_test_d3_t4, dataloader_test_d2_t4, dataloader_test_d1_t4]
dataname_dict = {dataloader_test_d1_t4:"data1", dataloader_test_d2_t4:"data2", dataloader_test_d3_t4:"data3", dataloader_test_d4_t4:"data4"}
task4_dataloader_ls = [dataloader_test_d4_t4]

for num,dataloader in enumerate(task4_dataloader_ls):
    print(f'Dataloader {num} start:\n')
    output_dict = {}
    responses = []
    for step,batch in tqdm(enumerate(dataloader)):
        with torch.no_grad():
            try:
                response, history = model.chat(tokenizer, batch, history=[], do_sample=False, temperature=1.0, top_p=1.0, num_beams=1, max_new_tokens=100,) #greedy search
                torch.cuda.empty_cache()
            except:
                response = "OOM:" + str(step)
            responses.append(response)
            output_dict[step] = response
    # save output_dict to json file
    output_dir = f'{output_folder}task{str(task)}/{dataname_dict[dataloader]}/'
    os.makedirs(output_dir, exist_ok=True)
    with open(output_dir+f'{dataloader.dataset.model_name}_OS.json', 'w') as f:
        json.dump(output_dict, f, indent=4, ensure_ascii=False)
    print(f'{dataloader.dataset.model_name}_OS.json finished')

# SFT template

- 单人单罪：本院认为，根据上述事实被告人xxx，其行为已触犯《中华人民共和国刑法》第xxx条，xxx条之规定，已经构成xxxx罪，应当判处被告人xxx年有期徒刑。


- 单人多罪：本院认为，根据上述事实被告人xxx犯有多罪，应当数罪并罚。
    在xxx罪案情中，被告人的行为已触犯《中华人民共和国刑法》第xxx条，xxx条之规定，判处有期徒刑xxx月；
    在xxx罪案情中，被告人的行为已触犯《中华人民共和国刑法》第xxx条，xxx条之规定，判处有期徒刑xxx月；
    ...
    综上，被告人xxx行为触犯数罪，应当数罪并罚决定执行有期徒刑xxx月。

- 多人单罪：本院认为，根据上述事实在被告人xx、xx中，被告人xxx其行为已触犯《中华人民共和国刑法》第xxx条，xxx条之规定，已经构成xx罪，应当判处被告人xxx年有期徒刑。

- 多人多罪：【本院认为，根据上述事实在被告人xx、xx中，被告人xxx犯有多罪，应当数罪并罚。
    在xxx罪案情中，被告人的行为已触犯《中华人民共和国刑法》第xxx条，xxx条之规定，判处有期徒刑xxx月；
    在xxx罪案情中，被告人的行为已触犯《中华人民共和国刑法》第xxx条，xxx条之规定，判处有期徒刑xxx月；
    ...
    综上，被告人xxx行为触犯数罪，应当数罪并罚决定执行有期徒刑xxx月。